# 9. 물체를 분리하자! - 세그멘테이션 살펴보기

그동안 다룬 image classfication은 물체의 종류를 분류하고, object detection은 물체의 존재와 위치를 탐지한다.   
이번에 배워볼 segmentation은 픽셀 수준에서 이미지의 각 부분이 어떤 의미를 갖는지 분리해내는 방법이다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/portrait_mode.max-800x600.jpg)

그림을 보면 segentation 기술을 쉽게 이해할 수 있다.   
쉽게 말하면 __이미지 분할__이다. 사진의 사람 영역과 배경 영역을 분리하고, 배경에 블러처리를 하여 아웃포커싱 효과를 보여주고 있다.

segmentation은 이미지의 영역을 어떤 영역인지 분리하는 기술로 다양한 곳에서 활용되고 있다. 핸드폰의 인물사진 모드가 대표적인 딥러닝 기반 기술이 숨어있는 예시이다.   
이러한 segmentation을 알아보자.

# 9-2. 세그멘테이션 문제의 종류

segmentation 모델들은 다양한 분야에서 활용되고 있다. 자율주행 카메라에 적용되어 이미지를 해석하기도 하고, 의료 이미지에서 병이 있는 영역을 판별해 내기도 한다.

이미지 내에서 영역을 분리하는 접근 방식은 크게 두 가지 방식이 있다. __semantic segmentation__, __instance segmentation__이다.   
영역을 분리하는 관점은 비슷하지만, 접근 방식에 따라 문제의 정의와 모델을 구성하는 방식이 달라진다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/semantic_vs_instance.png)

좌측 상단 사진은 양들이 길에 있는 사진에서 물체들의 localization과 classification을 보는 접근법이다.   
우측 상단은 개별 양들의 개체 하나하나 위치를 정확하게 식별하는 object detection 접근법이다.

좌측 하단은 양, 길, 풀밭의 영역을 정확히 구분해내고 있다. 이러한 접근 방식을 semantic segmentation이라고 한다.   
우측 하단은 양의 영역을 세세하게 개체 단위로 구분하고 있다. 이것을 instance segmentation이라고 한다.

### 1) sementic segmentation
![](https://d3s0tskafalll9.cloudfront.net/media/images/segnet_demo.max-800x600.png)

![](https://d3s0tskafalll9.cloudfront.net/media/images/leaf_segmentation.max-800x600.png)

![](https://d3s0tskafalll9.cloudfront.net/media/images/u-net.max-800x600.png)

위 사진들은 시맨틱 세그멘테이션의 활용 사례들이다.   
그리고 시맨틱 세그멘테이션의 대표적인 모델인 U-Net의 구조이다. 입력으로 572x572 크기의 이미지가 들어가고 출력으로 388x388의 크기에 두 가지 클래스를 가진 segmentation map이 나온다. 이는 가장 마지막 레이어의 채널 개수가 2라는 점에서 확인할 수 있다.

마지막에 나온 두 가지의 클래스를 문제에 따라 다르게 정의하면, 클래스에 따른 sementic segmentation map을 얻을 수 있다.   
인물사진 모드라면 사람과 배경 클래스를, 의료 인공지능에서는 세포 사진에서 병이 있는 영역과 정상인 영역 등을 지정해 적용할 수 있다.

이전에 다뤄봤던 모델들과 비교하면 input, output의 이미지 scale이 비교적 큰데, 이는 segmentation을 위해 각 픽셀에 해당하는 영역의 클래스별 정보가 필요하기 때문이다.

### 2) instance segmentation
인스턴스 세그멘테이션은 같은 클래스 내에서도 각 개체(instance)들을 분리하여 세그멘테이션을 수행한다. 하지만 어떻게 접근하면 좋을까?   
object detection 모델로 각 개체를 구분하고 이후에 개체별 sementic segmentation을 수행하면 괜찮지 않을까?

이러한 방식 중 대표적인게 Mask R-CNN이다. 2017년에 발표한 Mask-R-CNN은 2-stage object detection의 가장 대표적인 Faster-R-CNN을 계승했다.

Faster-R-CNN의 아이디어인 Region of Interest Pooling Layer(RoIPool) 개념을 개선하여 정확한 segmentation에 유리하게 한 RolAlign와 클래스별 마스크 분리라는 두 가지 아이디어를 적용했다.   
이는 클래스별 object detection과 sementic segmentation을 하나의 task로 엮어낸 것으로 평가받는 중요한 모델이다.

Faster-R-CNN 계열에서 사용하는 RoIPool 개념과 그 한계점에 대해 알아보자.   
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/GC-5-L-FasterRCNN-01.png)

RoIPool Layer는 다양한 RoI 영역을 Pooling을 통해 동일한 크기의 feature map으로 추출하는 레이어이다. 이후 이 고정 사이즈의 feature map을 바탕으로 바운딩 박스와 object 클래스를 추론한다.

이 구성은 object detection에서는 크게 문제가 없다. 하지만 object 영역의 정확한 마스킹을 필요하는 sementation에서 문제가 된다.   

![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-5-L-RoIPool-01.max-800x600.png)

RoIPool 과정에서 Quantization이 필요하다는 점이다. 그림의 예시에서는 16x16으로 분할했다. 이미지에서 RoI 영역의 크기는 다양한데, 모든 RoI 영역의 가로, 세로 픽셀의 크기가 16배수인 것은 아니다.   
그래서 가로 200, 세로 145 픽셀짜리 RoI 영역을 16x16으로 분할된 영역 중 절반 이상이 RoI 영역에 덮이는 곳들로 끼워 맞추다 보면, 영역 밖이 포함되는 경우도 있고, 자투리 영역이 버려지는 경우도 생긴다.   
이는 시맨틱 세그멘테이션의 정보손실과 왜곡을 야기하게 된다.   
위 그림에서 3x3 RoIPool을 적용했을 때 초록색은 불필요한 정보가 들어있고, 하늘색과 파란색은 정보를 잃어버리게 된다.

Mask-R-CNN의 RoIAlign은 RoI를 처리할 고정 사이즈의 Feature map을 생성할 수 있게 아이디어를 제공한다.   
__핵심 아이디어__는 RoI 영역을 pooling layer 크기에 맞춰 등분한 후, RoIPool을 했을 때 quantization 영역 중 가까운 것들과 bilinear interpolation 계산을 통해 생성해야 할 feature map을 계산해낸다는 점이다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/instance_seg.max-800x600.png)

Mask R-CNN은 Faster R-CNN에서 추출방식을 RoIAlign 방식으로 개선을하고 세그멘테이션을 더한 방식이다. 사진의 Faster R-CNN과 비교한 구조도를 보면 U-Net처럼 feature map의 크기를 키워 mask를 생성하는 부분을 통해 인스턴스에 해당하는 영역인 인스턴스 맵을 추론한다.   
Mask R-CNN은 클래스에 따른 mask를 예측할 때, 여러 가지 task를 한 모델로 학습하여 물체 검출의 성능을 높인다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/mask-rcnn-head.max-800x600.png)

그림을 보면 bounding box regression을 하는 Bbox head와 마스크를 예측하는 Mask Head 두 갈래로 나뉘고 있다.   
Mask map의 경우 시맨틱 세그멘테이션과 달리 상대적으로 작은 28x28의 특성 맵 크기를 갖는다. RoIAlign을 통해 줄어든 특성에서 마스크를 예측하기 때문에 정확한 마스크를 얻으려는 경우에는 부적합할 수 있다.

# 9-3. 주요 세그멘테이션 모델(1) FCN

세그멘테이션을 위한 대표적인 방법을 몇 가지 배워보자.   
세그멘테이션도 목적에 따라 오랫동안 여러 가지 모델이 발전했다.

시간 순서대로 Fully Convolutional Networks for Semantic Segmentation 논문의 FCN(Fully Convolutional Network)부터 확인해보자.

- [FCN 논문 리뷰 — Fully Convolutional Networks for Semantic Segmentation](https://medium.com/@msmapark2/fcn-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0-fully-convolutional-networks-for-semantic-segmentation-81f016d76204)
- [원본 논문: Fully Convolutional Networks for Semantic Segmentation](https://arxiv.org/abs/1411.4038)

![](https://d3s0tskafalll9.cloudfront.net/media/images/fcn.max-800x600.png)

FCN은 이미지넷 챌린지에서 좋은 성적을 거둔 AlenNet, VGG-16 등의 모델을 세그멘테이션에 맞게 변형한 모델이다.   
기본 VGG 모델은 이미지의 특성을 추출하기 위한 네트워크의 뒷단에 fully connected layer를 붙여 계산한 클래스별 확률을 바탕으로 이미지에 분류를 수행한다.   
FCN은 세그멘테이션을 하기 위해 네트워크 뒷단에 FC layer 대신 CNN을 붙인다.

fully connected layer 대신 CNN을 붙이는 이유가 무엇일까?   
CNN은 이미지 내 위치의 특성을 유지한다. 하지만 FC layer는 그렇지 않다. 위치정보를 유지하면서 클래스 단위의 heat map을 얻어 segmentation을 하기 위해 그렇게 했다.

CNN은 어떤 세팅을 갖게 될까?   
위치의 특성을 유지하면서 이미지 분류를 하기 위해 마지막 CNN은 1x1의 커널 크기와 클래스의 개수만큼 채널을 갖는다. 이렇게 CNN을 거쳐 클래스 히트맵을 얻을 수 있다.

히트맵의 크기는 일반적으로 원본 이미지보다 작다. CNN과 pooling 레이어를 거치면서 크기가 줄어들기 때문인데, 이를 키워주기 위해 upsampling을 해야한다.   
Upsampling도 여러가지가 있는데 FCN에서는 deconvolution과 interpolation 방식을 활용한다.

Deconvolution은 컨볼루션 연산을 거꾸로 해준 것이라 볼 수 있고, Interpolation은 보간법으로 주어진 값들을 통해 추정해야 하는 픽셀을(여기서 특성 맵의 크기가 커지면서 메꾸어야 하는 중간 픽셀을 의미한다.) 추정하는 방법이다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/fcn_3.max-800x600.png)

Upsampling만 하면 원하는 세그멘테이션 맵을 얻을 수 있다. FCN-32s가 그런 경우이다.

하지만 논문에서는 더 나은 성능을 위해 한 가지 기법을 더해준다. __Skip Architecture__ 방법인데, 논문에서는 FCN-32s, FCN-16s, FCN-8s로 결과를 구분해서 설명한다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/fcn_4.max-800x600.png)

FCN-16s는 앞쪽 블록에서 얻은 예측 결과 맵과, 2배로 upsampling한 맵을 더한 후, 한 번에 16배로 upsampling을 해주어 얻는다. 여기서 한 번 더 앞쪽 블록을 사용하면 FCN-8s를 얻을 수 있다.

# 9-4. 주요 세그멘테이션 모델(2) U-Net

- [U-Net 논문 리뷰 — U-Net: Convolutional Networks for Biomedical Image Segmentation](https://medium.com/@msmapark2/u-net-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0-u-net-convolutional-networks-for-biomedical-image-segmentation-456d6901b28a)
- [원본 논문: U-Net: Convolutional Networks for Biomedical Image Segmentation](https://arxiv.org/pdf/1505.04597.pdf)

![](https://d3s0tskafalll9.cloudfront.net/media/images/u-net.max-800x600.png)

U-Net은 이름에서 볼 수 있듯이 네트워크 구조가 U자 형태를 띠고 있다.   
__FCN에서 upsampling을 통해 특성 맵을 키운 것을 입력값과 대칭적으로 만든것이다.__

특이한 점은 U-Net이 세그멘테이션뿐만 아니라 여러가지 이미지 task에서 사용되는 유명한 네트워크가 되었지만, 의학 관련 논문으로 시작되었다는 점이다.   
논문 제목인 biomedical image segmentation을 통해 알 수 있듯이, 세포 영역을 세그멘테이션 하기 위해 개발된 방법이다.

### 전체 구조
논문에서 네트워크 구조를 좌측 contractiong path, 우측 expansive path 두 가지로 구분했다.

contracting path는 일반적으로 우리가 사용해온 convolution network와 유사한 구조를 가진다. 각 블록은 두 개의 3x3 convolution 계층과 ReLU를 가지고 그 뒤로 downsampling을 위해 2x2 커널을 2stride로 max pooling을 하게 된다. Downsampling을 거친 후 다음 convolution의 채널 크기는 두 배씩 늘어나도록 설계되었다.

Expansive path에서는 각 블록에 2x2 up-convolution이 붙어 채널이 절반씩 줄어들고 특성 맵의 크기는 늘어난다. Expansive path의 블록은 contracting block과 동일하게 3x3 convolution이 두 개씩 사용되었다.

두 Path에서 크기가 같은 불록의 입, 출력은 skip connection처럼 연결하여 low-level의 feature를 활용할 수 있도록 했다.   
마지막은 1x1 convolution으로 원하는 시맨틱 세그멘테이션 맵을 얻을 수 있다.

결과적으로 입력으로 572x572 크기인 이미지가 들어가고 출력으로 388x388의 크기에 두 가지의 클래스를 가진 segmentation map이 나온다.

마지막 segmentation map 크기가 입력 이미지와 다른 것은 세그멘테이션 맵을 원하는 크기로 resize하여 해결할 수 있다. 원본 이미지에 맞게 크기를 조정해주면 위에서 본 우리가 원하는 시맨틱 세그멘테션 결과를 얻을 수 있게 된다.

U-Net의 논문 구현을 기준으로 구조는 대칭이지만, 모델의 입출력값의 크기가 다르다.   
convolution은 padding을 통해 크기를 갖게 할 수 있지만, U-Net에서는 padding을 하지 않는다.

### Tile 기법
![](https://d3s0tskafalll9.cloudfront.net/media/images/unet.max-800x600.png)

U-Net이 downsampling과 upsampling을 대칭으로 하는 점은 구조에서 한눈에 파악이 된다.

또 다른 차이로 얻을 수 있는 세그멘테이션 맵의 해상도가 다르다. FCN은 입력 이미지의 크기를 조정하여 segmentation map을 얻었다. 하지만 U-Net은 tile(타일) 방식을 사용해서 어느 정도 서로 겹치는 구간으로 타일을 나누워 네트워크를 추론, 큰 이미지에서도 높은 해상도의 세그멘테이션 맵을 얻을 수 있도록 한다.

### 데이터 불균형 해결
![](https://d3s0tskafalll9.cloudfront.net/media/images/unet_2.max-800x600.png)

세포를 검출해내기 위해 세포의 영역 뿐만 아니라 경계 또한 예측을 해야한다. 이때 픽셀 단위로 라벨을 매기면, 데이터셋에 세포나 배경보다 절대적으로 세포 간 경계 면적이 작다.   
이러한 클래스 간 데이터 양의 불균형을 해결해주기 위해 분포를 고려한 weight map을 학습에 사용했다고 한다.

weight map의 weight를 신경망의 학습 파라미터를 가리키는 weight랑 다르다. 여기서 말하는 weight는 __손실 함수에 적용되는 가중치__를 말한다. 의료 영상에서 세포 내부나 배경보다 상대적으로 면적이 작은 세포 경계를 명확하게 추론하는게 중요하기 때문에, 세포 경계의 손실에 더 많은 패널티를 부과하는 방식이다.

# 9-5. 주요 세그멘테이션 모델(3) DeepLab 계열

- [Lunit 기술블로그의 DeepLab V3+: Encoder-Decoder with Atrous Separable Convolution for Semantic Image Segmentation](https://blog.lunit.io/2018/07/02/deeplab-v3-encoder-decoder-with-atrous-separable-convolution-for-semantic-image-segmentation/)
- [hyunjulie님의 2편: 두 접근의 접점, DeepLab V3+](https://medium.com/hyunjulie/2%ED%8E%B8-%EB%91%90-%EC%A0%91%EA%B7%BC%EC%9D%98-%EC%A0%91%EC%A0%90-deeplab-v3-ef7316d4209d)
- [원본 논문: Encoder-Decoder with Atrous Separable Convolution for Semantic Image Segmentation](https://arxiv.org/pdf/1802.02611.pdf)

DeepLabv3+는 이름에서 예측할 수 있듯이 이전의 많은 버전을 거쳐 개선을 이룬 네트워크이다. 처음 DeepLab 모델이 제안된 뒤 개선하기 위해 Atrous Convolution, Spatial Pyramid Pooling 등 많은 방법들이 제안됐다.   
DeepLabv3+의 전체 구조를 본 뒤 Dilated Convolution이라 불리는 Atrous Convolution, Spatial Pyramid Pooling을 살펴보자.

### 전체 구조
![](https://d3s0tskafalll9.cloudfront.net/media/images/deeplab_v3.max-800x600.png)

DeepLabV3+의 구조 그림이다.   
U-Net의 좌, 우를 나눈 Contracting path와 Expansive path의 역할을 하는게 그림에서 Encoder, Decoder이다.

인코더는 이미지에서 필요한 정보를 특성으로 추출하는 모듈이고, 디코더는 추출된 특성을 이용해서 원하는 정보를 예측하는 모듈이다.   
3x3 convolution을 사용한 U-Net과 다르게 DeepLabV3+는 Atrous Convolution을 사용하고 있다. 그리고 이로 Atrous Convolution을 여러 크기에 다양하게 적용하는게 ASPP(Atrous Spatial Pyramid Pooling)이다. DeepLab V3+는 ASPP가 있는 블록을 통해 특성을 추출하고 디코더에서 Upsampling을 통해 세그멘테이션 마스크를 얻고 있다.

### Atrous Convolution
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/atrous_conv_2.gif)

Atrous Convolution을 쉽게 말하면 띄엄띄엄 보는 컨볼루션이다.   
일반적인 컨볼루션과 달리 더 넓은 영역을 보기 위해 커널이 일정 간격으로 떨어져있다. 이를 통해 컨볼루션 레이어를 너무 깊게 쌓지 않아도 넓은 영역의 정보를 커버할 수 있다.

### Spatial Pyramid Pooling
![](https://d3s0tskafalll9.cloudfront.net/media/images/GC-5-L-SPP.max-800x600.png)

여러가자ㅣ 스케일로 convolution과 pooling을 하고 나온 다양한 특성을 concatenate 해준다. 이를 통해 멀티스케일로 특성을 추출하는 것을 병렬로 수행하는 효과를 얻을 수 있다. 여기서 컨볼루션을 Atrous Convolution으로 바꾸어 적용한 것은 Atrous Spatial Pyramid Pooling이라 한다. 이러한 아키텍쳐는 입력 이미지의 크기와 관계 없이 동일한 구조를 활용할 수 있는 장점이 있다. 그래서 다양한 크기와 비율을 가진 RoI 영역에 적용하기 유리하다.

# 9-6. 세그멘테이션의 평가
![](https://d3s0tskafalll9.cloudfront.net/media/images/segmentation_metric.max-800x600.png)

시맨틱 세그멘테이션을 수행하는 다양한 방법들을 알아보았으니, 이렇게 만들어진 모델을 평가하는 평가 지표를 알아보자.

시맨틱 세그멘테이션의 결과값은 이미지의 크기에 맞는 세그멘테이션 맵 크기와 시멘틱 클래스 수에 맞는 채널 크기를 갖는다. 여기서 각 채널의 max probability에 따라 해당 위치의 클래스가 결정된다.

### 1) 픽셀별 정확도(Pixel Accuracy)
![](https://d3s0tskafalll9.cloudfront.net/media/images/error_metric.max-800x600.jpg)

pixel accuracy는 픽셀에 따른 정확도를 의미한다.

세그멘테이션 문제를 픽셀에 따른 이미지 분류 문제로 생각했을 때, 이미지 분류 문제와 비슷하게 픽셀별 분류 정확도를 세그멘테이션 모델을 평가하는 기준으로 생각할 수 있다.

이때 예측 결과 맵(prediction map)을 클래스 별로 평가하는 경우 이진 분류 문제로 생각해 픽셀 및 채널 별로 평가한다.   
픽셀 별 이미지 분류 문제로 평가하는 경우는 픽셀 별로 정답 클래스를 맞췄는지 여부를 True, False로 구분한다.

예를 들어보자. 4x4 크기의 map에서 중앙의 2x2 영역이 전경이고 예측 결과 중 한 칸을 놓쳤다. 이때 위에서 보이는 Error Metrics를 확인하면 (TP+TN/FP+FN+FP+TN)으로 Accuracy를 구할 수 있다. TP+TN는 옳게 분류된 샘플의 수로 잘못 예측된 한 칸을 제외한 15이다.   
그리고 False case는 1인 한 칸은 전경이 배경으로 예측되었으니 FN이다. 따라서 분모항은 16이 된다. 그래서 Pixel Accuracy는 15/16으로 계산할 수 있다.

### 2) 마스크 IoU(Mask Intersection over Union)

물체 검출 모델을 평가할 때 정답 라벨(ground truth)과 예측 결과 바운딩 박스(prediction bounding box) 사이의 IoU를 사용한다.   
마스크도 일종의 영역임을 생각하면 세그멘테이션 문제는 정답인 영역과 예측한 영역의 IoU를 계산할 수 있다.

```
# sample for mask iou
intersection = np.logical_and(target, prediction)
union = np.logical_or(target, prediction)
iou_score = np.sum(intersection) / np.sum(union)
```
이렇게 세그멘테이션 마스크의 IoU를 계산할 수 있다.

마스크 IoU를 클래스별로 계산하면 한 이미지에서 여러 클래스에 대한 IoU 점수를 얻을 수 있다. 이를 평균하면 전체적인 시맨틱 세그멘테이션 성능을 가늠할 수 있다.

# 9-7. Upsampling의 다양한 방법

segmenation에서는 마스킹 단계에 upsampling이 중요하게 사용된다. 방법들을 알아보자.

### 1) Nearest Neighbor
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/upsampling1.png)

scale을 키운 위치에서 원본에서 가장 가까운 값을 그대로 적용하는 방법이다.   
그림을 예로 들자면 2x2 matrix에서 2배로 키우면 4x4 matrix가 되는데, 죄측 상단의 2x2 입력은 1이다. 그래서 이 값을 그대로 사용하게 된다.

### 2) Bilinear Interpolation
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/bi_interpolation.png)

두 축에 대해 선형보간법을 통해 필요한 값을 메우는 방식이다.   
2x2 matrix를 4x4로 upsampling 할 때 위 이미지처럼 빈 값들을 채워야한다. 이에 선형보간법(linear interpolation)을 사용하는것이다.   
축을 두 방향으로 활용하기 때문에 bilinear interpolation이라 한다.

그림을 보면 두 가지 interpolation을 적용한 것을 순서대로 확인할 수 있다.   
$R_1$이 $Q_11, Q_21$의 x축 방향 interpolation 결과이다.   
$R_1, R_2$을 y축 방향으로 interpolation하면 새로운 위치 $P$의 값을 추정할 수 있다.   
이러한 선형 보간 방법으로 interpolation 하는 방법이 있고, bicubic unterpolation의 경우 삼차보간법을 사용한다.

### 3) Transposed Convolution
![](https://d3s0tskafalll9.cloudfront.net/media/images/transposed_conv.max-800x600.jpg)

학습할 수 있는 파라미터를 가진 upsampling 방법 중 trasposed convolution이 있다.   
우리가 사용해온 convolution layer는 kernel의 크기를 정의하고 입력된 feature를 window에 따라 output을 계산한다.   

Transposed Convolution은 이와 반대의 연산을 한다. 따라서 거꾸로 학습된 파라미터로 입력된 벡터를 통해 더 넓은 영역의 값을 추정해낸다.